## HOUSING IN IRELAND

#### Loading Libraries:

In [176]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

In [150]:
os.getcwd()

'C:\\Users\\leand'

#### The data is sourced from propery price register.ie

In [151]:
data=pd.read_csv('Property_Price_Register_Ireland.csv')

In [152]:
data.shape

(476745, 9)

In [153]:
data.head()

,SALE_DATE,ADDRESS,POSTAL_CODE,COUNTY,SALE_PRICE,IF_MARKET_PRICE,IF_VAT_EXCLUDED,PROPERTY_DESC,PROPERTY_SIZE_DESC
0,2010-01-01,"5 Braemor Drive, Churchtown, Co.Dublin",NaN,Dublin,343000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
1,2010-01-03,"134 Ashewood Walk, Summerhill Lane, Portlaoise",NaN,Laois,185000.0,0,1,New Dwelling house /Apartment,greater than or equal to 38 sq metres and less...
2,2010-01-04,"1 Meadow Avenue, Dundrum, Dublin 14",NaN,Dublin,438500.0,0,0,Second-Hand Dwelling house /Apartment,NaN
3,2010-01-04,"1 The Haven, Mornington",NaN,Meath,400000.0,0,0,Second-Hand Dwelling house /Apartment,NaN
4,2010-01-04,"11 Melville Heights, Kilkenny",NaN,Kilkenny,160000.0,0,0,Second-Hand Dwelling house /Apartment,NaN


In [154]:
data.dtypes

SALE_DATE              object
ADDRESS                object
POSTAL_CODE            object
COUNTY                 object
SALE_PRICE            float64
IF_MARKET_PRICE         int64
IF_VAT_EXCLUDED         int64
PROPERTY_DESC          object
PROPERTY_SIZE_DESC     object
dtype: object

#### Converting the date column to its proper datetime type

In [155]:
data['SALE_DATE']=pd.to_datetime(data['SALE_DATE'])

In [156]:
data.dtypes

SALE_DATE             datetime64[ns]
ADDRESS                       object
POSTAL_CODE                   object
COUNTY                        object
SALE_PRICE                   float64
IF_MARKET_PRICE                int64
IF_VAT_EXCLUDED                int64
PROPERTY_DESC                 object
PROPERTY_SIZE_DESC            object
dtype: object

#### As post code and property description columns are mosly empty,ignore them

In [157]:
data.isnull().sum()

SALE_DATE                  0
ADDRESS                    0
POSTAL_CODE           386981
COUNTY                     0
SALE_PRICE                 0
IF_MARKET_PRICE            0
IF_VAT_EXCLUDED            0
PROPERTY_DESC              0
PROPERTY_SIZE_DESC    423953
dtype: int64

In [158]:
data=data.iloc[:,[0,1,3,4,5,6,7]]

#### Define functions to trim the address and property description to be more concise and readable.

In [159]:
TRIM_PROPERTY = lambda x: re.search(r"^([\S]+)", x).group()
TRIM_ADDRESS= lambda x: re.split(",", x)[-1]

In [160]:
data['PROPERTY_DESC']=data.iloc[:,6].apply(TRIM_PROPERTY)


In [161]:
data['ADDRESS']=data.iloc[:,1].apply(TRIM_ADDRESS)

#### Create a mortgage interest rate column,set to 1 for now

In [162]:
data['MORTGAGE_RATE']=1

In [163]:
data

,SALE_DATE,ADDRESS,COUNTY,SALE_PRICE,IF_MARKET_PRICE,IF_VAT_EXCLUDED,PROPERTY_DESC,MORTGAGE_RATE
0,2010-01-01,Co.Dublin,Dublin,343000.0,0,0,Second-Hand,1
1,2010-01-03,Portlaoise,Laois,185000.0,0,1,New,1
2,2010-01-04,Dublin 14,Dublin,438500.0,0,0,Second-Hand,1
3,2010-01-04,Mornington,Meath,400000.0,0,0,Second-Hand,1
4,2010-01-04,Kilkenny,Kilkenny,160000.0,0,0,Second-Hand,1
...,...,...,...,...,...,...,...,...
476740,2021-05-28,MULLINGAR,Westmeath,150000.0,0,0,Second-Hand,1
476741,2021-05-28,BORRIS,Carlow,170000.0,1,0,Second-Hand,1
476742,2021-05-28,WESTMEATH,Westmeath,175000.0,0,0,Second-Hand,1
476743,2021-05-28,GLENGARIFF,Cork,240000.0,0,0,Second-Hand,1


#### Mapping the year of sale to the average mortgage lending rate for tha year, mortgage rates have been obtained from the 

#### CENTRAL STATISTICS OFFICE IRELAND CSO.ie website

In [86]:
def map_interest_rate(year):
    mapping = {
        2010 : 4.02,
2011 : 4.42,
2012 : 4.33,
2013 : 4.38,
2014 : 4.20,
2015 : 4.05,
2016 : 3.61,
2017 : 3.44,
2018 : 3.21,
2019 : 3.02,
2020 : 2.92,
2021 : 2.62,
    }
    
    return mapping.get(year, "nothing")

In [165]:
data['MORTGAGE_RATE']=data['SALE_DATE'].dt.year.apply(map_interest_rate)

In [169]:
data

,SALE_DATE,ADDRESS,COUNTY,SALE_PRICE,IF_MARKET_PRICE,IF_VAT_EXCLUDED,PROPERTY_DESC,MORTGAGE_RATE
0,2010-01-01,Co.Dublin,Dublin,343000.0,0,0,Second-Hand,4.02
1,2010-01-03,Portlaoise,Laois,185000.0,0,1,New,4.02
2,2010-01-04,Dublin 14,Dublin,438500.0,0,0,Second-Hand,4.02
3,2010-01-04,Mornington,Meath,400000.0,0,0,Second-Hand,4.02
4,2010-01-04,Kilkenny,Kilkenny,160000.0,0,0,Second-Hand,4.02
...,...,...,...,...,...,...,...,...
476740,2021-05-28,MULLINGAR,Westmeath,150000.0,0,0,Second-Hand,2.62
476741,2021-05-28,BORRIS,Carlow,170000.0,1,0,Second-Hand,2.62
476742,2021-05-28,WESTMEATH,Westmeath,175000.0,0,0,Second-Hand,2.62
476743,2021-05-28,GLENGARIFF,Cork,240000.0,0,0,Second-Hand,2.62


#### Gathering additional variables like the median household income and deprivation rate for that year as these affect purchasing 

#### power, data gathered from CSO.IE

In [170]:
income_data=pd.read_csv('Income and Poverty Rates.csv')
income_data.head()


,Year,Median Real Household Disposable Income,Mean Real Household Disposable Income,Median Nominal Household Disposable Income,Mean Nominal Real Disposable Income,Median Equivalised Real Disposable Income,Mean Equivalised Real Disposable Income,Median Equivalised Nominal Disposable Income,Mean Equivalised Nominal Disposable Income,At Risk of Povery Rate,Deprivation Rate,Consistent Poverty Rate
0,2019,"46,255","55,660","47,373","57,005","26,005","29,613","26,634","30,329",7.3,10.3,1.8
1,2008,"45,233","52,973","45,401","53,170","22,347","26,067","22,430","26,164",11.4,8.6,2.3
2,2009,"42,093","50,078","42,093","50,078","22,397","25,400","22,397","25,400",10.1,9.5,2.5
3,2010,"41,262","49,388","39,801","47,640","21,418","25,085","20,660","24,197",11.5,14.8,3.4
4,2011,"40,711","47,598","39,819","46,555","21,159","24,276","20,695","23,744",10.6,17.2,3.8


In [171]:
income_data=income_data.loc[:,['Year','Median Real Household Disposable Income','Deprivation Rate']]



In [172]:
income_data.columns=['Year','Median income','Deprivation rate']

In [173]:
data['Year']=data['SALE_DATE'].dt.year
data['Month']=data['SALE_DATE'].dt.month_name()

In [174]:
data=data[data['Year']<2021]


#### Merging the two data frames to include median income and the deprivation rate

In [145]:
final_data=pd.merge(data, 
 income_data,
 how='inner',
 on='Year')

## FINAL DATA:

In [175]:
final_data.head()

,SALE_DATE,ADDRESS,COUNTY,SALE_PRICE,IF_MARKET_PRICE,IF_VAT_EXCLUDED,PROPERTY_DESC,MORTGAGE_RATE,Year,Month,Median income,Deprivation rate
0,2010-01-01,Co.Dublin,Dublin,343000.0,0,0,Second-Hand,4.02,2010,January,"41,262",14.8
1,2010-01-03,Portlaoise,Laois,185000.0,0,1,New,4.02,2010,January,"41,262",14.8
2,2010-01-04,Dublin 14,Dublin,438500.0,0,0,Second-Hand,4.02,2010,January,"41,262",14.8
3,2010-01-04,Mornington,Meath,400000.0,0,0,Second-Hand,4.02,2010,January,"41,262",14.8
4,2010-01-04,Kilkenny,Kilkenny,160000.0,0,0,Second-Hand,4.02,2010,January,"41,262",14.8
